In [1]:
import pandas as pd
import numpy as np
import os
from joblib import dump, load
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, recall_score, precision_score, roc_auc_score, f1_score
import xgboost as xgb
import lightgbm as lgb
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve

In [3]:
import os
import joblib
import pandas as pd
import numpy as np
from sklearn.metrics import (
    confusion_matrix, recall_score, accuracy_score, precision_score,
    f1_score, roc_auc_score, roc_curve
)

# 定义模型路径列表和故障名称映射
model_paths = ['./Models/1001', './Models/2001', './Models/4001', './Models/4002', './Models/4003', './Models/5001', './Models/5002', './Models/6001', './Models/6002']
fault_mapping = {
    '1001': '物料推送装置故障1001',
    '2001': '物料检测装置故障2001',
    '4001': '填装装置检测故障4001',
    '4002': '填装装置定位故障4002',
    '4003': '填装装置填装故障4003',
    '5001': '加盖装置定位故障5001',
    '5002': '加盖装置加盖故障5002',
    '6001': '拧盖装置定位故障6001',
    '6002': '拧盖装置拧盖故障6002'
}

# 定义评估函数
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    specificity = tn / (tn + fp)
    sensitivity = recall_score(y_test, y_pred)
    g_mean = np.sqrt(specificity * sensitivity)
    fpr, tpr, thresholds = roc_curve(y_test, y_prob)
    metrics = {
        'Accuracy': accuracy_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'F1-Score': f1_score(y_test, y_pred),
        'AUC': roc_auc_score(y_test, y_prob),
        'G-Mean': g_mean
    }
    return metrics

# 初始化结果列表
results = []

# 加载和评估每个模型
for model_path in model_paths:
    folder_name = os.path.basename(model_path)
    fault_name = fault_mapping[folder_name]  # 使用映射来获取完整的故障名称
    df_test = pd.read_csv('./Datasets/附件1/M101.csv')  # 读取测试数据集
    # 假设 df_test 是你的测试数据集
    selected_columns = ['时间', '物料推送气缸推送状态', '物料推送气缸收回状态', '物料推送数', '物料待抓取数',
                    '放置容器数', '容器上传检测数', '填装检测数', '填装定位器固定状态', '填装定位器放开状态', 
                    '物料抓取数', '填装旋转数', '填装下降数', '填装数', '加盖检测数', '加盖定位数', 
                    '推盖数', '加盖下降数', '加盖数', '拧盖检测数', '拧盖定位数', '拧盖下降数', 
                    '拧盖旋转数', '拧盖数', '合格数', '不合格数']

    # 将目标列作为 y
    df_test[fault_name] = df_test[fault_name].apply(lambda x : 0 if x == 0 else 1)
    y_test = df_test[fault_name].to_numpy()
    
    # 从测试集中提取所选列
    df_test_selected = df_test[selected_columns]
    X_test = df_test_selected

    for model_file in os.listdir(model_path):
        if model_file.endswith('.joblib'):
            full_path = os.path.join(model_path, model_file)
            model = joblib.load(full_path)
            metrics = evaluate_model(model, X_test, y_test)
            
            # 从文件名解析模型类型和数据处理方法，并添加模型路径或故障名称
            model_type, data_handling = model_file.split('_')[:2]
            
            metrics['Model'] = model_type
            metrics['Data Handling'] = data_handling
            metrics['Model File'] = model_file
            metrics['Fault Name'] = fault_name  # 添加故障名称
            metrics['Model Path'] = model_path  # 添加模型路径
            results.append(metrics)

# 将结果转换为DataFrame
results_df = pd.DataFrame(results)

# 将结果保存到Excel文件中
results_df.to_excel('./models/model_evaluation_summary.xlsx', index=False)

print("模型评估结果已经保存到Excel文件中。")

C:\Users\Tunan\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Tunan\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Tunan\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\User

模型评估结果已经保存到Excel文件中。
